In [17]:
# UNIVERSIDADE ESTADUAL DE CAMPINAS
# FT - UNICAMP / CURSO SISTEMAS DE INFORMAÇÃO
#TCC - CORRETOR ORTOGRÁFICO PARA ANALISE DE TIPOS DE ERROS E SUA RELAÇÃO COM O ANALFABETISMO
# AUTOR - MATHEUS EDUARDO DA SILVA RA: 230719
# ORIENTADORA -ANA ESTELA ANTUNES DA SILVA

In [18]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup 
import nltk
#nltk.download('punkt')
import os
import pandas as pd 
from nltk.test.portuguese_en_fixt import setup_module
import nltk.corpus

#lendo o conjunto de corpus C2, e salvando seu conteudo em all_texts.txt:
parser = ET.XMLParser(encoding="UTF-8")
file = open("all_texts.txt", "w", encoding="UTF-8")

directory = 'corpus02/ResDialCorpus/C2/'
for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        f = os.path.join(directory, filename)
        tree = ET.parse(f)
        root = tree.getroot()
        text = root.findtext('text')
        file.write(text)

file.close()

In [19]:
#split_words(): recebe uma lista de strings, e cria um novo vetor que contem somente palavras válidas(sem caracteres nao alfanumericos) 
def split_words(tokens_list):
    list_words = []
    for token in tokens_list:
        if token.isalpha():
            list_words.append(token)
    return list_words

#registrando palavras dos resumos e historias
with open('all_texts.txt', encoding='UTF-8') as f:
    contents = f.read()

#registrando palavaras do dicionario em ptbr
with open('dicionario_USP_ptBr.txt', encoding='UTF-8') as f:
    dict_content = f.read()

#tokenizando e criando a lista de palavras dos 2 grupos de palavras(dicionário e resumos)
contents = nltk.tokenize.word_tokenize(contents)
dict_content = nltk.tokenize.word_tokenize(dict_content)
folha_sp = nltk.corpus.mac_morpho.words()

list_words = split_words(contents)
list_words_dict = split_words(dict_content)
list_words_dict_common_words_folhasp = split_words(folha_sp)
print(f"Número de palavras(resumos) é {len(list_words)}")
print(f"Número de palavras(dicionario PTBR) é {len(list_words_dict)}")
print(f"Número de palavras(dicionario Folha SP) é {len(list_words_dict_common_words_folhasp)}")

Número de palavras(resumos) é 60678
Número de palavras(dicionario PTBR) é 261798
Número de palavras(dicionario Folha SP) é 1012661


In [20]:
#normalizando as palavras(tornando todas em minúsculas):
def normalization_words(list_words):
    normalized_list = []
    for word in list_words:
        normalized_list.append(word.lower())
    return normalized_list

normalized_list = normalization_words(list_words)
normalized_list_dict = normalization_words(list_words_dict)
normalized_list_dict_folhaSP = normalization_words(list_words_dict_common_words_folhasp)

In [21]:
def create_database_words_analysis(list_words_normalized, list_words_dict_normalized):
    database = []
    for word in list_words_normalized:
        if word in list_words_dict_normalized:
            line = [word, word]
            database.append(line)
        else:
            line = [word, ""]
            database.append(line)

    return database

#lista que relaciona as palavras do resumo com o dicionário(REMOVER PALAVRAS REPETIDAS)
database_words_analysis = create_database_words_analysis(normalized_list, normalized_list_dict)
print(database_words_analysis)


KeyboardInterrupt: 

In [ ]:
#cirando um dataframe com relação de palavras conhecidas e desconhecidas
df_dictionary = pd.DataFrame(database_words_analysis, columns=['Palavra', 'Dicionario(palavra referente)'] )
df_dictionary

,Palavra,Dicionario(palavra referente)
0,o,o
1,cliente,cliente
2,da,da
3,lanchonete,lanchonete
4,quer,quer
...,...,...
60673,irrita,irrita
60674,com,com
60675,o,o
60676,pedido,pedido


In [ ]:
#numero de palavras desconhecidas ao dicionario
database_words_analysis[1].count("")
count_unknown_words = df_dictionary.loc[df_dictionary['Dicionario(palavra referente)'] ==''].count()
count_unknown_words

Palavra                          3942
Dicionario(palavra referente)    3942
dtype: int64

In [ ]:
#FAZER O TESTE DO CORRETOR ORTOGRÁFICO
#FAZER UMA RELAÇAO COM PALAVRAS DESCONHECIDAS ("META PARA CORRIGIR")
#CRIAR TIPOS DE CLASSIFICACAO(ERROS DE ASPECTOS VISUAIS E GRAFICOS + ERROS DE RELAÇÕES ENTRE LETRAS E SONS( VARIAS SUBCLASSES))

# -----> AQUI PODEM TER n CLASSES CASO QUEIRA SER ESPECIFICO
#COMPARAR COM O TRABALHO E VER SE OS NÚMEROS ESTÃO BATENDO
#COMPARAR E FAZER ANALISE GRÁFICA

In [ ]:
frequency = nltk.FreqDist(normalized_list_dict_folhaSP)
words_total = len(normalized_list_dict_folhaSP)
frequency.most_common(10)

[('de', 88320),
 ('a', 68966),
 ('o', 61303),
 ('em', 34383),
 ('e', 22551),
 ('que', 20666),
 ('os', 16989),
 ('para', 11122),
 ('as', 10720),
 ('por', 10311)]

In [31]:
import re
from collections import Counter


# TO DO 
#checar funcionalidade dos error groups
#checar como retornar palavra já correta
#checar palavra correta onde é igual a original (NORVIG)
#criar base textual
#criar avaliador

def known_words(words): 
    lista = set(w for w in words if w[0] in normalized_list_dict)
    return lista


#Geração de palavras candidatas a possibilidades de correção
def candidates_words(word):
    candidates = known_words([word]) | known_words(edits_distance_1(word)) | set([word])
    return candidates
    #return (known_words([word]) or (known_words(edits_distance_1([word]))) or [word])

#Probabilidade da palavra ser a correte dentre as possíveis:
    #usar contexto do texto da Folha SP
def probability(word): 
    if isinstance(word, tuple) == True:
        return (frequency[word[0]] /words_total)
    else:
        return (frequency[word]/words_total)


#CORRIGIR ISSO AQUI 
#Maior probabilidade de correção da palavra
def correction(word):
    result = max(candidates_words(word), key=probability) #, default=word)
    if isinstance(result, tuple) != True:
        return (word, "")
    else:
        return (result)




#EDIT WORDS POSSIBILITIES
    #GRUPOS DE ERROS:
    #1- Emprego das consoantes e dos dígrafos + Emprego das formas que representam o som nasal
    #2- Emprego de vogais
    #3- Acréscimo e omissão de letras
    #4- Inversão de letras
    #5- Letras com formato semelhante
    #6- Erros decorrentes de escritas particulares + Segmentação indevida das palavras
    #7- Uso de acentuação

def edit_word_insert(word_sliced, letters):
    error_group = 3
    error_group_content = []
    new_possible_words = []
    for E, D in word_sliced:
        for letter in letters:
            new_possible_words.append(E + letter + D)
            error_group_content.append(error_group)
    words = list(zip(new_possible_words, error_group_content))
    
    return words

def edit_word_delete(word_sliced, letters):
    error_group = 3
    error_group_content = []
    new_possible_words = []
    for E, D in word_sliced:
        if len(D) > 0:
            new_possible_words.append(E + D[1:])
            error_group_content.append(error_group)
    words = list(zip(new_possible_words, error_group_content))

    return words

def edit_word_transpose(word_sliced, letters):
    error_group = 4
    error_group_content = []
    new_possible_words = []
    for E, D in word_sliced:
        if len(D) > 1:
            new_possible_words.append(E + D[1] + D[0] + D[2:])
            error_group_content.append(error_group)
    words = list(zip(new_possible_words, error_group_content))

    return words

#corrgir - existe insercao no for de substituicao(ultima letra extra)
def edit_word_replace(word_sliced, letters):
    error_group = 0 # POR PADRAO SER ESCRITA ESPECIFICA

    error_group_content = []
    new_possible_words = []
    vogals = ['a', 'e', 'i', 'o', 'u']
    digrafos = ['ch', 'lh', 'nh', 'rr', 'ss', 'sc', 'sç', 'xc', 'xs', 'am', 'an', 'em', 'en', 'im', 'in', 'om', 'on', 'um', 'un', 'gu', 'qu'] #gu e qu somente nos casos de fonema único(sem som "/u/")
    
    letters_equals_p = ['p', 'f', 'q'] #melhorar as opções aqui
    letters_equals_m = ['m', 'n']

    accents = ['á', 'â','à','ã','é', 'ê', 'è', 'ẽ', 'í','î', 'ì','ĩ' 'ó', 'ô', 'õ', 'ò', 'ú', 'û', 'ù', 'ũ', 'ç']
    for E, D in word_sliced:
        if len(D) > 0:
            for letter in letters:
                removed_letter = D[:1]
                new_possible_words.append(E + letter + D[1:]) #letra D[0] é substituida
                if len(E) > 1 and len(D) > 1:
                    possible_digraphs = [(E[-1] + letter), (letter + D[1])]
                elif len(D) > 1:
                    possible_digraphs = [(E + letter), (letter + D[1])]
                elif len(E) > 1:
                    possible_digraphs = [(E[-1] + letter), (letter + D)]
                else:
                    possible_digraphs = [(E + letter), (letter + D)]

                if ((possible_digraphs[0] in digrafos) or (possible_digraphs[1] in digrafos)) and (letter != removed_letter): #se soma letra + letra anterior ou proxima resulta em digrafo
                    error_group = 1 #digrafos, consoantes ou sons nasais
                elif ((removed_letter in letters_equals_p and letters in letters_equals_p) or (removed_letter in letters_equals_m and letters in letters_equals_m)) and (letter != removed_letter): #erros de formato semelhante
                    error_group = 5
                elif (((removed_letter not in vogals) and (letter not in vogals) and (letter not in accents) and (removed_letter not in accents))) and (letter != removed_letter): #consoantes incorretas
                    error_group = 1 #digrafos, consoantes ou sons nasais
                elif ((removed_letter in vogals) and (letter in vogals)) and (letter != removed_letter): #vogais incorretas
                    error_group = 2 #uso de vogais
                elif ((removed_letter in vogals) and (letter in accents)) or (removed_letter == 'c' and letter == 'ç') and (letter != removed_letter): #erros de acentuacao
                    error_group = 7 #erro de acentuação
                else:
                    error_group = 6 #erros de escritas particulares
                error_group_content.append(error_group)
    words = list(zip(new_possible_words, error_group_content))

    return words 


#distâncias de edição para as palavras no corretor
def edits_distance_1(word):
    word_sliced = []
    letters = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
    for i in range(len(word)+1):
        word_sliced.append((word[:i], word[i:]))

    deletes    = edit_word_delete(word_sliced, letters)
    #print(deletes)
    inserts    = edit_word_insert(word_sliced, letters)
    #print(inserts)
    transposes = edit_word_transpose(word_sliced, letters)
    #print(transposes)
    replaces   = edit_word_replace(word_sliced, letters)
    #print(replaces)

    return set(deletes + transposes + replaces + inserts)


In [29]:
probability("familia") #CORRIGIR
#print(probability('água') > probability('agua'))

0.00017972450800415933

In [36]:
correction("eses") # Corrigir grupos de erro 1, 2, 5, 6 e 7 palavra água

('esse', 4)

In [ ]:
'https://www.nltk.org/howto/portuguese_en.html'

'https://docs.huihoo.com/nltk/0.9.5/guides/portuguese_en.html'

'https://direct.mit.edu/coli/article/41/1/175/1493/Spelling-Error-Patterns-in-Brazilian-Portuguese'

' http://rbep.inep.gov.br/ojs3/index.php/rbep/article/view/1506/1245'

'https://www.nltk.org/howto/portuguese_en.html'
'https://www.nltk.org/nltk_data/' # MAC Morpho

#wordnet -> criar/usar uma lista de palavras(dicionário) - check
#.txt -> criar uma lista de palavras - 
#criar um .csv com a relação palavra do txt x dicionario(atraves do corretor?)
#criar um .txt para base de teste (200 250 palavras erradas + correta + classificação de correção)
#dividir o .txt anterior para base de treino e teste
#criar o classificador ortográfico de erros sobre analfabetismo(estudar e pensar nos subgrupos) - check 1/2
#fazer analytics com os dados

'https://www.nltk.org/nltk_data/'

In [ ]:
#criador de dados de teste: (a alterar e adaptar)
def create_test_dataset(file):
    list_words_test = []
    correct = ''
    wrong = ''
    error_group = 0
    f = open(file, "r", encoding='UTF-8')
    for line in f:
        correct, wrong, error_group = line.split(" ", 3)
        list_words_test.append((correct, wrong, error_group))
    f.close()
    return list_words_test

#lista_teste = create_test_dataset("palavras.txt")

In [ ]:
#ajustar formato das saidas do corretor:
def adjust_format(word):
    if isinstance(word, tuple) != True:
        return (word, "" )
    else:
        return word

#avaliador do corretor ortográfico: ( corrigir para mostrar as porcentagens de tipo de erro)
def evalutate_corrections(tests, vocabulary):
    number_words = len(tests)

    right_corrections_right_group = 0
    right_corrections_wrong_group = 0
    wrong_corrections_right_group = 0
    wrong_corrections_wrong_group = 0
    unknown = 0
    for correct, wrong, error_group in tests:
        error_group = error_group[0]
        #corrected_word: word, error_group(made)
        corrected_word = correction(wrong)
        corrected_word = adjust_format(corrected_word)
        unknown += (correct not in vocabulary)

        if (str(corrected_word[0]) == correct) and (str(corrected_word[1]) == str(error_group)):
            right_corrections_right_group += 1 
        elif (str(corrected_word[0]) == correct) and (str(corrected_word[1]) != str(error_group)):
            right_corrections_wrong_group += 1 
            print("Word(wrong/correction/right):" + wrong + " - "  + str(corrected_word[0]) + " - " + correct + " *  Groups(wrong/right): " + str(corrected_word[1]) + "-" + error_group)
        elif (str(corrected_word[0]) != correct) and (str(corrected_word[1]) == str(error_group)):
            wrong_corrections_right_group += 1
            print("Word(wrong/correction/right):" + wrong +  " - " + str(corrected_word[0]) + " - " + correct + " *  Groups(wrong/right): " + str(corrected_word[1]) + "-" + error_group)
        else:
            wrong_corrections_wrong_group += 1
            print("Word(wrong/correction/right):" + wrong  + " - " + str(corrected_word[0]) + " - " + correct + " *  Groups(wrong/right): " + str(corrected_word[1]) + "-" + error_group)

    accuracy_percentage = round(right_corrections_right_group*100/number_words, 2)
    accuracy_percentage_error_group = round((right_corrections_right_group + wrong_corrections_right_group)*100/number_words, 2)
    unknown_percentage = round(unknown*100/number_words, 2)
    print(f"{accuracy_percentage}% de {number_words} palavras, desconhecida é {unknown_percentage}%,  acurácia de erros de grupo {accuracy_percentage_error_group}%")

In [37]:
words_testing_list = create_test_dataset('words_testing.txt')
evalutate_corrections(words_testing_list, normalized_list_dict_folhaSP)

Word(wrong/correction/right):esje - este - esse *  Groups(wrong/right): 1-1
Word(wrong/correction/right):eme - em - ele *  Groups(wrong/right): 3-1
Word(wrong/correction/right):quaôdo - quando - quando *  Groups(wrong/right): 1-6
Word(wrong/correction/right):sjava - soava - java *  Groups(wrong/right): 6-3
Word(wrong/correction/right):eòtá - está - está *  Groups(wrong/right): 6-3
Word(wrong/correction/right):tim - tem - tem *  Groups(wrong/right): 1-2
Word(wrong/correction/right):ptyhon - ptyhon - python *  Groups(wrong/right): -4
Word(wrong/correction/right):assem - assim - assim *  Groups(wrong/right): 1-2
Word(wrong/correction/right):te - de - ter *  Groups(wrong/right): 1-3
Word(wrong/correction/right):ebm - em - bem *  Groups(wrong/right): 3-4
Word(wrong/correction/right):desigm - desigm - design *  Groups(wrong/right): -5
Word(wrong/correction/right):apemas - apenas - apenas *  Groups(wrong/right): 1-5
Word(wrong/correction/right):emqresa - empresa - empresa *  Groups(wrong/righ

In [ ]:
33.18 + 5.07 39.63
68.2% de 217 palavras, desconhecida é 5.07%,  acurácia de erros de grupo 70.51%